# Preparación de datos
*"Es un error capital teorizar antes de tener datos. Sin darse cuenta, uno empieza a deformar los hechos para que se ajusten a las teorías, en lugar de ajustar las teorías a los hechos."  
Arthur Conan Doyle en "Escándalo en Bohemia".*

## Selección de filas y columnas

Airbnb es una plataforma en línea que permite a los usuarios alquilar alojamientos. El archivo "listings_cdmx.csv" contiene el listado de alojamientos en la Ciudad de México, con información básica actualizada al 25 de septiembre de 2024. Las variables disponibles son:
- id: identificador del anuncio de alojamiento.
- name: nombre del alojamiento.
- host_id: identificador del anfitrión.
- host_name: nombre del anfitrión
- neighbourhood_group:
- neighbourhood: alcaldía.
- latitude: latitud.	
- longitude: longitud.
- room_type: Puede ser "Entire place", "Private room", "Shared room" u "Hotel".
- price: Precio en moneda nacional ($ MXN).
- minimum_nights: mínimo de noches de estadía.
- last_review: fecha de la última reseña.
- reviews_per_month: promedio de reseñas por mes en el tiempo publicado.	
- calculated_host_listings_count: número de anuncios que tiene el anfitrión.
- availability_365: Dias disponibles en los siguientes 365 días.
- number_of_reviews_ltm: número de reseñas en los últimos 12 meses.
- license: número de licencia o registro

In [ ]:
# Importa la biblioteca de pandas
import pandas as pd

Para propósitos de este ejercicio solo usaremos algunas de las variables disponibles.

In [ ]:
# Carga de archivo
filepath = '../datasets/listings_cdmx.csv'
#filepath = 'https://github.com/adan-rs/AnalisisDatos/raw/main/datasets/listings_cdmx.csv'
columns = ["host_id", "neighbourhood", "room_type", "price", 
             "minimum_nights", "number_of_reviews_ltm", "license"]
original_data = pd.read_csv(filepath, usecols=columns)

Aunque es común utilizar `df` como nombre de un DataFrame, es recomendable utilizar nombres descriptivos que nos permitan distinguir si los datos son originales, seleccionados o procesados.

Revisa la información del dataframe con `.head()` y `.info()`

In [ ]:
original_data.info()

In [ ]:
original_data.head(5)

Observa que la variable *host_id* tiene valores numéricos no es una variable cuantitativa. La reclasificaremos como *object* para evitar que se hagan operaciones matemáticas

In [ ]:
original_data["host_id"] = original_data["host_id"].astype('object')
original_data.info()

La base de datos abarca diferentes tipos de hospedajes. Mediante `df['A'].value_counts()` podemos consultar qué categorías y frecuencias tiene una variable 'A' en un dataframe llamado 'df'.

In [ ]:
# Obtén la frecuencia por tipo de hospedaje
original_data['room_type'].value_counts()

Seleccionaremos solamente casos bajo dos condiciones específicas:
- Anuncios de casas o departamentos: `original_data['room_type'] == 'Entire home/apt'`
- Anuncios con al menos una reseña en el último año: `original_data['number_of_reviews_ltm'] > 0`
  
Si incluimos dos condiciones en un filtro debemos poner cada condición en un paréntesis:  
`mask = (original_data['room_type'] == 'Entire home/apt') & (original_data['number_of_reviews_ltm] > 0)`  
Ahora podemos utilizar:   
`original_data = original_data[mask]`  
o bien  
`original_data = original_data.loc[mask]`

In [ ]:
# Seleccionar casas o departamentos con reseñas recientes.
mask = (original_data['room_type'] == 'Entire home/apt') & (original_data['number_of_reviews_ltm'] > 0)
original_data = original_data.loc[mask]

Una buena práctica es trabajar sobre una copia para no modificar los datos originales

In [ ]:
# Crear copia del DataFrame original
df = original_data.copy()

In [ ]:
df

Todos los pasos anteriores del código se pueden encapsular en una función con un nombre descriptivo

In [ ]:
def load_and_filter(filepath):
    """
    Carga y limpieza de datos originales.
    """
    columns = ["host_id", "neighbourhood", "room_type", "price", 
               "minimum_nights", "number_of_reviews_ltm", "license"]
    data = pd.read_csv(filepath, usecols=columns)
    data["host_id"] = data["host_id"].astype('object')
    mask = (data['room_type'] == 'Entire home/apt') & (data['number_of_reviews_ltm'] > 0)
    return data.loc[mask].copy()  

In [ ]:
filepath = 'data/listings_cdmx.csv'
df = load_and_filter(filepath)

## Datos duplicados
En algunas ocasiones puede haber datos duplicados en nuestra base de datos. Para visualizar los datos duplicados podemos usar *duplicated()* de la siguiente manera:
`df[df.duplicated()]`. Si nos interesa en la variable "A" en particular entonces es `df[df['A'].duplicated()]`

In [ ]:
# Identifica los datos duplicados en todas las variables 
df[df.duplicated()]

Una opción para eliminar datos duplicados es usar *drop_duplicates()*. En nuestro ejemplo sería: `df.drop_duplicates()`. Esto sólo elimina observaciones filas duplicadas en todas las variables. De manera predeterminada, solamente conserva la primera fila de ellas.  
En algunas ocasiones necesitaremos eliminar observaciones duplicadas en solamente algunas variables. En ese caso se puede agregar un listado con el argumento *subset*, por ejemplo: `df = df.drop_duplicates(subset=['hotel_id'])`

In [ ]:
def remove_duplicates(df):
    """
    Elimina filas duplicadas en todas las columnas y registra la cantidad 
    de filas eliminadas.Retorna DataFrame sin valores duplicados.
    """
    rows_before = len(df)
    df = df.drop_duplicates()
    rows_after = len(df)
    print(f"Se eliminaron {rows_before - rows_after} filas duplicadas")
    return df

In [ ]:
df = remove_duplicates(df)

## Datos perdidos
En el caso de datos perdidos es importante determinar si se pueden considerar aleatorios o si existe un patrón que pueda afectar los resultados. En una encuesta, por ejemplo, es posible que las personas que prefieran no mencionar su ingreso tengan ciertas características, o bien, que las personas tiendan a no responder a preguntas sobre cierto tipo de comportamiento que no es socialmente aceptable.

Es recomendable realizar pruebas estadísticas para verificar si los datos perdidos pueden ser considerados aleatorios o no. Bajo el procedimiento habitual, se construyen dos grupos, uno de observaciones con datos completos y otro grupo de observaciones con datos perdidos en una variable en particular. Posteriormente, se realizan pruebas para comparar si existen diferencias significativas en los valores promedio de las otras variables

Como una aproximación inicial, el método `info()` puede servir para identificar qué variables tienen valores perdidos.

In [ ]:
# Identifica qué variables tienen valores perdidos con 'info()'
df.info()

Para encontrar las filas con datos perdidos podemos usar `df[df.isna().any(axis=1)]`. El método *isna()* sirve para indicar qué valores son perdidos y el método *any* sirve para indicar qué filas tienen al menos un valor perdido. Otra opción equivalente a `isna()` es `isnull()`

Para encontrar las filas con datos perdidos de una variable en particular (por ejemplo "price") podemos usar `df[df.price.isna()]`. El método isna() sirve para indicar qué valores son perdidos. Para encontrar todas las filas con datos perdidos podemos usar `df[df.isna().any(axis=1)]` donde el método any sirve para indicar qué filas tienen al menos un valor perdido. Otra opción equivalente a isna() es isnull()

In [ ]:
# Identificar filas con datos perdidos en una variable
df[df.price.isna()]

*¿Qué hacer con datos perdidos?*  
Los métodos más comunes para tratar con datos perdidos son:
- Borrar filas o columnas con datos perdidos. Si una columna tiene una gran cantidad de datos perdidos, considera excluir esa columna del análisis. Si pocas filas tienen datos perdidos (digamos, menos del 5%) y estos parecen ser completamente aleatorios, borrar las filas correspondientes es una alternativa. Sin embargo, en series de tiempo no es recomendable eliminar filas (*¿por qué?*)
- Codificar: Los datos perdidos también pueden aportar información en un análisis, por lo que en variables no numéricas se puede codificar los datos perdidos en una categoría adicional.
- Imputación simple: Reemplazar los datos perdidos con un valor calculado a partir de la misma variable. La opción más común y conservadora es utilizar el promedio. Sin embargo, este método es criticado debido a que reduce la variabilidad y afecta la estimación de los intervalos de confianza (Treiman, 2009).
- Imputación multivariada: Reemplazar los datos perdidos con un valor calculado a partir de otras variables. En la imputación de datos por medio de la regresión se utilizan otras variables en la base de datos para establecer una ecuación de regresión en la que la variable dependiente es la variable con los datos perdidos. La ecuación de regresión estimada se utiliza para estimar los datos perdidos 


En cuanto a otros métodos de estimación se recomienda la lectura del capítulo “Multiple imputation of missing data” de Treiman (2009).


*Imputación simple o univariada*  
La imputación con la media para una columna 'A' es:  
`df['A'] = df['A'].fillna(df['A'].mean())`  
En caso de querer la imputación con la mediana se reemplaza `mean()` con `median()` y en el caso de la moda se reemplaza con `mode().iloc[0]` 

In [ ]:
def handle_missing_data(df, threshold=0.2):
    """ 
    Maneja datos perdidos según el tipo de datos de cada columna, 
    y elimina columnas con un porcentaje de datos perdidos mayor a 
    un umbral específico. 
    Retorna DataFrame procesado sin valores nulos.
    """  
    print("\nValores faltantes por columna:")
    print(df.isnull().sum())
    
    # Remove columns with high proportion of missing values
    cols_with_nulls = df.isnull().mean() > threshold
    cols_to_drop = df.columns[cols_with_nulls]
    if len(cols_to_drop) > 0:
        print(f"\nColumnas eliminadas por tener más de {threshold*100}% de valores nulos:")
        print(cols_to_drop.tolist())
    df = df.drop(columns=cols_to_drop)
    
    # Handle missing values in remaining columns
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            df[col] = df[col].fillna(df[col].median())
        elif df[col].dtype == 'string':
            df[col] = df[col].fillna('DESCONOCIDO')
            
    total_remaining_nulls = df.isna().sum().sum()
    print(f"\nValores nulos restantes: {total_remaining_nulls}")
    
    return df

In [ ]:
df = handle_missing_data(df)

## Valores atípicos
Un valor atípico (outlier) es un valor extremo en una o más variables. En el caso de series univariadas, es común utilizar el método del valor z o el criterio del rango intercuartil para identificar valores atípicos. Para el caso de datos multivariados generalmente se utilizan criterios basados en la distancia de Mahalanobis y otras técnicas multivariadas. 

*¿Qué hacer con datos atípicos?*
- Si el valor extremo es un error de captura o parte de otra población lo recomendable es corregir o borrar el caso o variable.
- Si el valor extremo es parte de los datos que nos interesa analizar se debe mantener (p. ej. ventas en navidad).
- En algunas variables económicas se recomienda transformar la variable de forma tal que el valor extremo no impacte los resultados (p. ej. transformación logarítmica, transformación de Box-Cox o la recodificación de datos).


*Método del valor z*: En este enfoque un valor atípico es aquel que esté a más de tres desviaciones estándar a partir de la media.

In [ ]:
def remove_outliers_3s(df, column):
    """
    Elimina valores atípicos utilizando el método del valor z. 
    Retorna un DataFrame sin valores atípicos.
    """
    mean = df[column].mean()
    std = df[column].std()
    upper_limit = mean + 3 * std
    lower_limit = mean - 3 * std
    df_clean = df[(df[column] > lower_limit) & (df[column] < upper_limit)]
    excluded_values = len(df) - len(df_clean)
    print(f"Cantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
remove_outliers_3s(df,'price')

*Método del rango intercuartil*. Otro criterio común es considerar como atípicos los valores que están a más de 1.5 veces el rango intercuartil hacia el extremo a partir del 1er. o 3er. cuartil.

In [ ]:
def remove_outliers_iqr(df, column):
    """
    Elimina valores atípicos utilizando el criterio del rango intercuartil.
    Retorna DataFrame sin valores atípicos.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    df_clean = df[(df[column] > lower_limit) & (df[column] < upper_limit)]
    excluded_values = len(df) - len(df_clean)
    print(f"Cantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
remove_outliers_iqr(df, 'price')

*Algoritmo isolation forest*: Es un algoritmo utilizado para detectar valores atípicos en grandes conjuntos de datos. Divide los datos de manera aleatoria y cuanto más rápido se aísla un punto mayor es la probabilidad de que sea un valor atípico (anomalía)

In [ ]:
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"\nCantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

Seleccionaremos este último criterio para filtrar datos atípicos

In [ ]:
df = remove_outliers_iso_forest(df, columns=['price', 'minimum_nights'])

## Reporte general

In [ ]:
def generate_quality_report(df):
    """
    Genera un reporte básico de calidad de datos que incluye
        - Dimensiones del DataFrame
        - Tipos de datos por columna
        - Cantidad de valores únicos por columna
        - Estadísticas descriptivas para columnas numéricas
    """
    print("\nREPORTE DE CALIDAD DE DATOS")
    print("-" * 50)
    
    # Basic information
    print(f"Dimensiones del DataFrame: {df.shape}")
    print(f"\nTipos de datos:")
    print(df.dtypes)
    
    # Unique values per column
    print("\nValores únicos por columna:")
    for column in df.columns:
        unique_count = df[column].nunique()
        print(f"{column}: {unique_count} valores únicos")
    
    # Basic statistics for numeric columns
    print("\nEstadísticas básicas:")
    print(df.describe().round(4))

In [ ]:
# Generar reporte de calidad de datos
generate_quality_report(df)

## Exportar datos (opcional)

Un dataframe lo podemos guardar con `df.to_excel('archivo.xlsx', index=False)`

In [ ]:
# Exporta el dataframe depurado con el nombre 'output'
# df.to_excel('output.xlsx', index=False)

## Resumen del flujo de trabajo

In [ ]:
# Archivo de origen
filepath = '../datasets/listings_cdmx.csv'

# Selección de datos
df = load_and_filter(filepath)
    
# Eliminar filas duplicadas
df = remove_duplicates(df)
    
# Manejar datos perdidos
df = handle_missing_data(df)

# Eliminar valores atípicos
df = remove_outliers_iso_forest(df, columns=['price', 'minimum_nights'])

## Ejercicio
En un nuevo notebook, utiliza las funciones creadas y el flujo de trabajo para procesar los datos de Airbnb en otra ciudad. Por ejemplo, reemplaza el filepath por `'../datasets/listings_madrid.csv'` o el enlace al archivo original:  
`'https://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2024-09-11/visualisations/listings.csv'`

## Referencias
- Una discusión interesante sobre el tratamiento de datos se puede encontrar en: Treiman, D. J. (2009). *Quantitative data analysis. Doing social research to test ideas*. San Francisco, CA: Jossey-Bass.
- La base de datos fue tomada de https://insideairbnb.com/get-the-data/ para fines no comerciales.